In [ ]:
# 所有的头

In [1]:
import torch # for save
import os
from van import Fan
from localconfig import config
import datetime
import random
from pprint import pprint
import html
from pyltp import SentenceSplitter
from pyltp import Postagger
from pyltp import Segmentor
from pyltp import NamedEntityRecognizer
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import Image
from IPython.core.display import HTML 
import pyecharts
from pyecharts import Pie
from pyecharts import Line
import reverse_geocoder as rg

%matplotlib inline

plt.rcParams['font.sans-serif']=['FangSong'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号


In [2]:
# 所有的配置

In [3]:
keyword = '贸易战' # 关注的关键词
until_year = 2000 # 最早的年份
until_num = 10000 # 截止的饭否数量
timeout = (60,60) # 超时时间

In [4]:
# 登录
fan = Fan(config['Consumer key'], config['Consumer secret'])
fan.xauth(config['my username'], config['my password'])

{'oauth_token': '1405515-071ad3666b5af9441d5cbb0dc1d3985b',
 'oauth_token_secret': '43cc293f5d127423f960b8636688577b'}

In [5]:
# 本小时内可用API次数
fan.request('GET', 'account/rate_limit_status', {}, timeout=timeout)

{'reset_time': 'Mon Mar 18 22:41:52 +0000 2019',
 'remaining_hits': 150,
 'hourly_limit': 150,
 'reset_time_in_seconds': 1552948912}

In [11]:
# 采集关键词消息
def datetimefromstr(s):
    return datetime.datetime.strptime(s, '%a %b %d %H:%M:%S %z %Y')
def get_status(keyword, until_year, until_num):
    latest = fan.request('GET', 'search/public_timeline', {'q':keyword, 'count':60}, timeout=timeout)
    
    while True:
        year = datetimefromstr(latest[-1]['created_at']).year
        if len(latest) >= until_num:
            break
        if year < until_year:
            break
        print(len(latest), year,'/', until_num, until_year, end='\r')
        len_ = len(latest)
        latest += fan.request('GET', 'search/public_timeline', { 'q':keyword, 'count':60, 'max_id':latest[-1]['id']}, timeout=timeout)[1:]
        if len(latest) == len_:
            break
    return latest

In [12]:
# 下载并暂存
statuses = get_status(keyword, until_year, until_num)

In [13]:
torch.save(statuses,'tmp_statuses.torch.pkl')

In [14]:
# 恢复
# statuses = torch.load('tmp_statuses.torch.pkl')

In [15]:
print(len(statuses))

1519


In [16]:
# common
def get_top(l, num=5, reverse=False): # 从大到小
    cnt = {}
    for x in l:
        if type(x) == tuple and len(x) == 3:
            x = x[0]
        if x not in cnt:
            cnt[x] = 0
        cnt[x] += 1
    cnt_list = [(k,cnt[k],cnt[k]/len(l)) for k in cnt]
    cnt_list = sorted(cnt_list, key=lambda x:x[1])[::-1]
    if reverse:
        cnt_list = cnt_list[::-1]
    if num == -1:
        return cnt_list
    return cnt_list[:num]
def huabing(y): # x [(label,,0.xxx)]
    attr = [x[0] for x in y]
    v1 = [x[2] for x in y]
    if sum(v1) < 1.0:
        attr.append('其他')
        v1.append(1.0 - sum(v1))
    pie = Pie('')
    pie.add('', attr, v1, is_label_show=True)
    return pie

In [17]:
# 时间段分布

In [18]:
print(datetimefromstr(statuses[-1]['created_at']))
print(datetimefromstr(statuses[0]['created_at']))
begin_year = datetimefromstr(statuses[-1]['created_at']).year
end_year = datetimefromstr(statuses[0]['created_at']).year

2008-04-30 08:59:45+00:00
2019-03-16 01:10:34+00:00


In [19]:
huabing(get_top([datetimefromstr(x['created_at']).year for x in statuses], num = -1))

In [21]:
arr = sorted(get_top([datetimefromstr(x['created_at']).year for x in statuses],num=-1),key = lambda x:x[0])
attr = [str(x[0]) for x in arr ]
v1 = [x[1] for x in arr ]
line = Line('')
line.add(keyword, attr, v1,is_smooth=True,is_label_show=True,mark_line=[ "average"])
# line.add("商家B", attr, v2, is_smooth=True, mark_line=["max", "average"])
line

In [22]:
# 用户信息文件
os.path.exists('./users.torch.pkl')

True

In [23]:
# 预处理用户信息
userid_to_info = {}
if os.path.exists('./users.torch.pkl'):
    userid_to_info = torch.load('users.torch.pkl')
for x in statuses:
    userid_to_info[x['user']['id']] = x['user']
torch.save(userid_to_info, 'users.torch.pkl')
print(len(userid_to_info))

5462


In [24]:
# 发表人数/总消息数量
uni_num = len(set([x['user']['id'] for x in statuses ]))
print(uni_num, len(statuses), uni_num/len(statuses))

658 1519 0.43317972350230416


In [26]:
# 发表top30的人
users_top = get_top([x['user']['id'] for x in statuses], num=30)
names_top = [ (userid_to_info[x[0]]['name'] ,  x[1], x[2],x[0]) for x in users_top]
pprint(names_top)
huabing(names_top)

[('牛山鎮出品', 51, 0.03357472021066491, '牛山镇出品'),
 ('财经频道', 34, 0.02238314680710994, 'cashmoney'),
 ('亥乃亚克山冬', 28, 0.018433179723502304, 'BillGateFun'),
 ('那.谁', 26, 0.017116524028966424, '海边的暴走族'),
 ('抱阳', 23, 0.015141540487162607, 'xxlzw'),
 ('MDJsoftware', 21, 0.013824884792626729, 'mdjsoftware'),
 ('华尔街日报', 19, 0.012508229098090849, '华尔街日报中文网'),
 ('吴下阿阿阿蒙', 18, 0.01184990125082291, '~koUOq8qty7I'),
 ('张晓张8586', 18, 0.01184990125082291, '~Vr29VIRGRcs'),
 ('安达充的走狗', 17, 0.01119157340355497, 'wylwddjz'),
 ('今天你开心吗', 15, 0.009874917709019092, 'fyw321'),
 ('时光云海', 15, 0.009874917709019092, 'songyuyao'),
 ('逸博', 15, 0.009874917709019092, 'king66lele'),
 ('一介大叔', 14, 0.009216589861751152, 'ningqun'),
 ('剑花', 14, 0.009216589861751152, 'dezhaos'),
 ('饭士', 13, 0.008558262014483212, '奋发图强'),
 ('小世界.', 12, 0.007899934167215274, 'uramazing'),
 ('Solibot', 11, 0.007241606319947334, 'solibot'),
 ('奔驰的回忆', 11, 0.007241606319947334, '奔驰的回忆'),
 ('笨不傻', 11, 0.007241606319947334, 'ngs882'),
 ('康村的村夫', 11,

In [ ]:
def show_user_info(x):
    user = userid_to_info[x]
    pprint(user)
    return Image(url = user['profile_image_url_large'])

In [ ]:
show_user_info(users_top[0][0])

In [28]:
# 被引用top20的消息

repost_top = get_top([x['repost_status']['id'] for x in statuses if 'repost_status' in x], num=20)
for i in range(len(repost_top)):
    x = repost_top[i]
    for y in statuses:
        if 'repost_status' in y and y['repost_status']['id'] == x[0]:
            repost_top[i] = ('id: '+y['repost_status']['user']['id'] + ' name: '+y['repost_status']['user']['name']  +' '+ html.unescape(y['repost_status']['text']),x[0], x[1],x[2])
            break
            
pprint(repost_top)

[('id: 海边的暴走族 name: 那.谁 未来的贸易战，贵国被打成日本也不是不可能，自求多福吧。',
  'B0MzT8DTMEw',
  14,
  0.03225806451612903),
 ('id: sexyfish name: sexyfish '
  '最近发现了一个节奏：贸易战-->国库告急-->催缴税款-->舆论围剿（各个灰色行业）--->税务出马-->巨额罚款',
  'XrHuLxwkGKU',
  14,
  0.03225806451612903),
 ('id: xxlzw name: 抱阳 '
  '如果不是因为一个叫特朗普的疯子发起的贸易战，我们大概率会陶醉在过去40年的“经济奇迹”与“大国崛起”的耀眼光环里，感觉良好，乃至一醉不起',
  'MavNsbPgbFY',
  12,
  0.027649769585253458),
 ('id: uramazing name: 小世界. '
  '下周贸易战2000亿出结果，如果美国对我们进行贸易战的话，在中国，就是这个经济发展的结束了。因为我们长期的发展，基本上都是开放人力，借助外力来发展的。如果美国跟我们打贸易战，增长型的经济不会出现了，那么只会出现所谓存量的经济的争夺战。存量经济的争夺本质上都是财富转移而已。',
  '06vPmkJNdNc',
  11,
  0.02534562211981567),
 ('id: wangchu name: Tennichy 中国已经在仍未结束的贸易战中赢了一百多次了。',
  'CAWAVl666pM',
  9,
  0.020737327188940093),
 ('id: 牛山镇出品 name: 牛山鎮出品 '
  '【转】2018年4月，湖南大学研究生辩论赛决赛，正方观点为“掀起中美贸易战是因为美国膨胀了”，反方观点是“掀起中美贸易战是因为美国心虚了”。--也不应该忽视其他可能啊，譬如“两者皆是”:-)',
  'DMERRcLwpSQ',
  8,
  0.018433179723502304),
 ('id: gravity0 name: gravity0 搜索出口退税（export tax '
  'refund），出来结果几乎都是关于天朝，貌似美帝没有相应政策。理论上如果天朝把增收的进口税全用于出口补贴，则贸易战对出

In [31]:
# 被引用top20的人
repost_user_id_top = get_top([x['repost_status']['user']['id'] for x in statuses if 'repost_status' in x], num=20)
repost_name_top =  [ (userid_to_info[x[0]]['name'] , x[1], x[2],  x[0]) for x in repost_user_id_top]
pprint(repost_name_top)

[('牛山鎮出品', 56, 0.12903225806451613, '牛山镇出品'),
 ('sexyfish', 21, 0.04838709677419355, 'sexyfish'),
 ('抱阳', 19, 0.04377880184331797, 'xxlzw'),
 ('那.谁', 17, 0.03917050691244239, '海边的暴走族'),
 ('小世界.', 16, 0.03686635944700461, 'uramazing'),
 ('剑花', 14, 0.03225806451612903, 'dezhaos'),
 ('康村的村夫', 13, 0.029953917050691243, '~-d7SWGD7osE'),
 ('Tennichy', 12, 0.027649769585253458, 'wangchu'),
 ('亥乃亚克山冬', 12, 0.027649769585253458, 'BillGateFun'),
 ('安达充的走狗', 9, 0.020737327188940093, 'wylwddjz'),
 ('王兴', 9, 0.020737327188940093, 'wangxing'),
 ('饭而不否', 9, 0.020737327188940093, 'dyxin'),
 ('gravity0', 8, 0.018433179723502304, 'gravity0'),
 ('慧淫禅师', 7, 0.016129032258064516, '~9EKpNhzk6yw'),
 ('二他姐姐', 7, 0.016129032258064516, 'znz28'),
 ('饭士', 6, 0.013824884792626729, '奋发图强'),
 ('瑞秋Rachel', 6, 0.013824884792626729, 'sugarball86'),
 ('一介大叔', 6, 0.013824884792626729, 'ningqun'),
 ('素合子', 5, 0.01152073732718894, 'suheti'),
 ('今天你开心吗', 5, 0.01152073732718894, 'fyw321')]


In [32]:
# top20 gender
gender_top = get_top([x['user']['gender'] or '未知' for x in statuses ], num=20)
pprint(gender_top)
huabing(gender_top)

[('男', 845, 0.5562870309414089),
 ('未知', 471, 0.31007241606319946),
 ('女', 203, 0.1336405529953917)]


In [ ]:
# converter = {
#     'Jiangsu Sheng' : '江苏',
#     'Shandong Sheng' : '山东',
#     'Shanghai Shi' : '上海',
#     'Beijing' : '北京',
#     'Fujian': '福建',
#     'Guangxi Zhuangzu Zizhiqu': '广西',
#     'Yunnan': '云南',
#     'Tianjin Shi': '天津',
#     'Zhejiang Sheng' : '浙江',
#     'Guangdong': '广东',
#     'Sichuan': '四川',
#     'Hebei': '河北',
#     'TW': '台湾',
#     'Liaoning': '辽宁',
#     'Hunan': '湖南',
#     'Changchun Shi': '长春',
#     'KR' : '韩国',
#     'Henan Sheng': '河南',
#     'Shanxi Sheng': '山西',
#     'Jilin Sheng': '吉林',
#     'Anhui Sheng': '安徽',
#     'Heilongjiang Sheng': '黑龙江',
#     'Jiangxi Sheng': '江西',
#     'ES': '西班牙',
#     'Hubei': '湖北',
#     'SJ' : '斯瓦尔巴群岛和扬马延岛',
#     'Shaanxi': '陕西',
#     'Gansu Sheng': '甘肃',
#     'HK': '香港',
#     'CA' : '加拿大',
# }
# torch.save(converter, 'geocoverter-en-cn.torch.pkl')

In [33]:
# top20 location

converter = torch.load('geocoverter-en-cn.torch.pkl')
def geotransform(x):
    if len(x) == 0:
        return '未知'
    if ',' in x and len(x.split(',')) == 2:
        try:
            a = x.split(',')
            results = rg.search((float(a[0]),float(a[1]))) # default mode = 2
            if len(results) == 0:
                return '未知'
            if results[0]['cc'] == 'CN':
                if results[0]['admin1'] in converter:
                    return converter[results[0]['admin1']]
                print('\n',len(results[0]['admin1']),results[0],'\n')
                return results[0]['admin1']
            else:
                if results[0]['cc'] in converter:
                    return converter[results[0]['cc']]
                print('\n',len(results[0]['cc']),results[0],'\n')
                return results[0]['cc']
        except Exception as e:
            print(x)
            return '未知'
    if ' ' in x and len(x.split(' ')) == 2:
        return x.split(' ')[0]
    return x
    
location_top = get_top([geotransform(x['location']) for x in tqdm(statuses)], num=20)
pprint(location_top)
huabing(location_top)

  0%|          | 0/1519 [00:00<?, ?it/s]

Loading formatted geocoded file...


100%|██████████| 1519/1519 [00:02<00:00, 733.56it/s]

[('未知', 535, 0.35220539828834757),
 ('北京', 174, 0.11454904542462147),
 ('广东', 128, 0.08426596445029624),
 ('上海', 108, 0.07109940750493746),
 ('美国', 63, 0.041474654377880185),
 ('山东', 50, 0.032916392363396975),
 ('江苏', 39, 0.025674786043449638),
 ('四川', 35, 0.02304147465437788),
 ('河南', 33, 0.021724818959842),
 ('浙江', 33, 0.021724818959842),
 ('辽宁', 25, 0.016458196181698487),
 ('重庆', 24, 0.015799868334430547),
 ('海外', 22, 0.014483212639894667),
 ('湖北', 22, 0.014483212639894667),
 ('黑龙江', 18, 0.01184990125082291),
 ('福建', 15, 0.009874917709019092),
 ('魔都', 13, 0.008558262014483212),
 ('404', 11, 0.007241606319947334),
 ('饭否刺客兄弟会', 11, 0.007241606319947334),
 ('台湾', 8, 0.005266622778143515)]


In [34]:
def remove_atag(s):
    if '<a' not in s:
        return s
    start = s.find('>')
    end = s.find('</a>',start)
    return s[start+1:end]

In [35]:
# top20 source
source_top = get_top([remove_atag(x['source']) for x in statuses ], num=20)
pprint(source_top)
huabing(source_top)

[('网页', 472, 0.3107307439104674),
 ('御飯 iOS', 233, 0.1533903884134299),
 ('猫饭', 133, 0.08755760368663594),
 ('蘑菇饭', 106, 0.06978275181040158),
 ('Android客户端', 68, 0.04476629361421988),
 ('尚饭', 67, 0.044107965766951945),
 ('手机上网', 59, 0.038841342988808425),
 ('有饭', 51, 0.03357472021066491),
 ('API', 42, 0.027649769585253458),
 ('饭否财经频道', 34, 0.02238314680710994),
 ('炒饭', 31, 0.02040816326530612),
 ('饭团fantuan', 26, 0.017116524028966424),
 ('享拍微博通', 25, 0.016458196181698487),
 ('小饭', 22, 0.014483212639894667),
 ('饭否分享', 21, 0.013824884792626729),
 ('泡饭', 18, 0.01184990125082291),
 ('FFReeder', 17, 0.01119157340355497),
 ('Twidere', 14, 0.009216589861751152),
 ('Mr.Greeting', 11, 0.007241606319947334),
 ('「御飯 iPhone」', 8, 0.005266622778143515)]


In [36]:
# nlp part

In [39]:
def myfilter(s):
    s = s.replace('<strong>','')
    s = s.replace('</strong>','')
    while '转@' in s:
        pos = s.find('转@')
        end = len(s)
        for endsym in [' ', ']', '：',':','@']:
            end2 = s.find(endsym,pos+2)
            if end2 == -1:
                end2 = len(s)
            end = min(end,end2)
        if end-pos > 15:
            print('alert [转@] too long',s)
            s = s[:pos] + s[pos+2:]
        else:
            s = s[:pos] + s[end:]
    while '@' in s:
        pos = s.find('@')
        end = len(s)
        for endsym in [' ', ']', '：',':','@', '\n']:
            end2 = s.find(endsym,pos+1)
            if end2 == -1:
                end2 = len(s)
            end = min(end,end2)
        if end-pos > 15:
            print('alert [@] too long',s)
            s = s[:pos] + s[pos+1:]
        else:
            s = s[:pos] + s[end:]
    if '@' in s:
        print('alert @ in ', s)
    return s

In [40]:
sentences = []
for x in tqdm(statuses):
    s = html.unescape(x['text']) 
    s = myfilter(s)
    if keyword not in s:
        continue
    if '</' in s:
        print(s)
    ss = SentenceSplitter.split(s)
    for t in ss:
        if len(t) > 4:
            sentences.append(t)
    

100%|██████████| 1519/1519 [00:00<00:00, 22371.47it/s]

alert [@] too long @小世界.跟一个国家一样，不是贸易战，我们会发生改变吗？人总是遇到外力，才会发生转变。没有外力的时候，自己怎么做都觉得是对的。只有外力的强加，才会有重新调整迭代的机会。有时候需要示弱，有时候需要示强。人一生总要经历，七灾八劫，这也是成长的阶梯，并且在这个磨砺的过程中，让自己调配到


In [41]:
print(len(sentences))

3317


In [42]:
# 对sentences去重。。 拒绝转发的影响 （存疑？）
# 不去重

In [43]:
# sentences = list(set(sentences))

In [44]:
print(len(list(set(sentences))))

2568


In [45]:
# torch.save(['尼玛', '卧槽', '傻逼', '对了', '有木有', '他妈'],'blacklist.torch.pkl' )
def add_to_blacklist(w):
    global blacklist
    blacklist = torch.load('blacklist.torch.pkl')
    blacklist.append(w)
    torch.save(blacklist,'blacklist.torch.pkl')

In [46]:
# 无意义
blacklist = torch.load('blacklist.torch.pkl')

In [47]:
words_all = []

segmentor = Segmentor()
postagger = Postagger()
recognizer = NamedEntityRecognizer()
postagger.load('./ltp_data_v3.4.0/pos.model')
segmentor.load('./ltp_data_v3.4.0/cws.model')
recognizer.load('./ltp_data_v3.4.0/ner.model')

#todo 情感分析

for s in tqdm(sentences):
    words = segmentor.segment(s)
    postags = postagger.postag(words) #词性标注
    netags = recognizer.recognize(words, postags) # 命名实体识别
    for i in range(len(words)):
        if words[i] in blacklist:
            continue
        words_all.append((words[i],postags[i],netags[i]))

segmentor.release()
postagger.release()
recognizer.release()

100%|██████████| 3317/3317 [00:08<00:00, 376.66it/s]


In [48]:
print(len(words_all))

65884


In [49]:
get_top([x[0] for x in words_all if len(x[0])>1 and x[1]!='wp'], num = 25)

[('贸易战', 1465, 0.04701238688145819),
 ('中国', 767, 0.02461331108401258),
 ('美国', 691, 0.02217444323214171),
 ('贸易', 385, 0.012354791091714267),
 ('经济', 259, 0.008311404916244143),
 ('我们', 242, 0.007765868686220397),
 ('可能', 161, 0.005166549001989602),
 ('因为', 160, 0.005134458635517617),
 ('一个', 157, 0.005038187536101663),
 ('如果', 123, 0.003947115076054168),
 ('没有', 117, 0.003754572877222258),
 ('影响', 111, 0.0035620306783903473),
 ('川普', 102, 0.0032732173801424814),
 ('现在', 101, 0.003241127013670496),
 ('这个', 98, 0.0031448559142545408),
 ('美元', 97, 0.003112765547782556),
 ('国家', 95, 0.0030485848148385855),
 ('什么', 92, 0.0029523137154226302),
 ('特朗普', 89, 0.002856042616006675),
 ('还是', 86, 0.0027597715165907193),
 ('出口', 86, 0.0027597715165907193),
 ('关税', 85, 0.0027276811501187343),
 ('欧盟', 83, 0.002663500417174764),
 ('新闻', 83, 0.002663500417174764),
 ('日本', 81, 0.002599319684230794)]

In [50]:
posdict = {}
nedict = {}
for x in tqdm(words_all):
    if x[1] not in posdict:
        posdict[x[1]] = []
    if x[2] not in nedict:
        nedict[x[2]] = []
    posdict[x[1]].append(x)
    nedict[x[2]].append(x)

100%|██████████| 65884/65884 [00:00<00:00, 875011.95it/s]


In [51]:
# 词性
print(posdict.keys())


dict_keys(['nt', 'v', 'u', 'q', 'n', 'wp', 'ni', 'd', 'nl', 'i', 'r', 'o', 'ns', 'j', 'm', 'a', 'nz', 'p', 'nh', 'nd', 'ws', 'c', 'b', 'e', 'z', 'k', 'h'])


In [52]:
# ['i'] 成语
pprint(get_top(posdict['i'], num=25))

[('一醉不起', 11, 0.026378896882494004),
 ('没什么', 11, 0.026378896882494004),
 ('亲朋好友', 9, 0.02158273381294964),
 ('一触即发', 8, 0.019184652278177457),
 ('说不定', 8, 0.019184652278177457),
 ('经济危机', 8, 0.019184652278177457),
 ('好不了', 7, 0.016786570743405275),
 ('两败俱伤', 6, 0.014388489208633094),
 ('他妈的', 6, 0.014388489208633094),
 ('惨绿', 5, 0.011990407673860911),
 ('急刹车', 5, 0.011990407673860911),
 ('风平浪静', 5, 0.011990407673860911),
 ('一厢情愿', 5, 0.011990407673860911),
 ('悬崖勒马', 4, 0.009592326139088728),
 ('对外开放', 4, 0.009592326139088728),
 ('一泻千里', 4, 0.009592326139088728),
 ('摩拳擦掌', 4, 0.009592326139088728),
 ('一败涂地', 4, 0.009592326139088728),
 ('就地取材', 4, 0.009592326139088728),
 ('不可避免', 4, 0.009592326139088728),
 ('畅通无阻', 3, 0.007194244604316547),
 ('空穴来风', 3, 0.007194244604316547),
 ('无心插柳', 3, 0.007194244604316547),
 ('闭关锁国', 3, 0.007194244604316547),
 ('自给自足', 3, 0.007194244604316547)]


In [53]:

# ['j'] 缩略词
pprint(get_top(posdict['j'], num=25))


[('美', 521, 0.354421768707483),
 ('中', 443, 0.3013605442176871),
 ('欧盟', 83, 0.05646258503401361),
 ('欧', 60, 0.04081632653061224),
 ('华', 26, 0.017687074829931974),
 ('川普', 21, 0.014285714285714285),
 ('房地产', 19, 0.01292517006802721),
 ('央行', 19, 0.01292517006802721),
 ('俄', 15, 0.01020408163265306),
 ('韩', 9, 0.006122448979591836),
 ('日', 9, 0.006122448979591836),
 ('微博', 8, 0.005442176870748299),
 ('中共', 8, 0.005442176870748299),
 ('经贸', 8, 0.005442176870748299),
 ('央视', 7, 0.004761904761904762),
 ('发改委', 7, 0.004761904761904762),
 ('世贸', 6, 0.004081632653061225),
 ('华大', 6, 0.004081632653061225),
 ('官媒', 6, 0.004081632653061225),
 ('川', 6, 0.004081632653061225),
 ('贸易战', 6, 0.004081632653061225),
 ('政协', 6, 0.004081632653061225),
 ('三反五反', 5, 0.003401360544217687),
 ('汽协', 4, 0.0027210884353741495),
 ('朝', 4, 0.0027210884353741495)]


In [54]:

# ['n'] 名词
pprint(get_top(posdict['n'], num=25))


[('贸易战', 1103, 0.09632346519954589),
 ('人', 263, 0.022967426425639682),
 ('经济', 259, 0.022618111955287748),
 ('国家', 95, 0.008296218670858441),
 ('关税', 85, 0.007422932494978604),
 ('新闻', 83, 0.007248275259802637),
 ('时候', 76, 0.006636974936686752),
 ('战', 74, 0.006462317701510785),
 ('国', 72, 0.006287660466334818),
 ('问题', 71, 0.006200331848746834),
 ('公司', 68, 0.005938345995982883),
 ('政府', 67, 0.0058510173783949),
 ('美帝', 63, 0.005501702908042965),
 ('国际', 59, 0.005152388437691031),
 ('大豆', 59, 0.005152388437691031),
 ('市场', 58, 0.005065059820103048),
 ('结果', 56, 0.004890402584927081),
 ('股', 52, 0.004541088114575147),
 ('人民币', 51, 0.004453759496987163),
 ('股市', 50, 0.0043664308793991795),
 ('企业', 49, 0.004279102261811196),
 ('全球', 47, 0.004104445026635229),
 ('产品', 47, 0.004104445026635229),
 ('世界', 46, 0.004017116409047245),
 ('战争', 44, 0.0038424591738712775)]


In [55]:

# ['nh'] 人名词
pprint(get_top(posdict['nh'], num=25))


[('特朗普', 28, 0.06526806526806526),
 ('奥巴马', 24, 0.055944055944055944),
 ('川普', 24, 0.055944055944055944),
 ('孟晚舟', 16, 0.037296037296037296),
 ('蛤蛤', 15, 0.03496503496503497),
 ('崔永元', 8, 0.018648018648018648),
 ('陈德铭', 7, 0.016317016317016316),
 ('崔天凯', 7, 0.016317016317016316),
 ('罗姆尼', 6, 0.013986013986013986),
 ('张首晟', 6, 0.013986013986013986),
 ('孙立平', 5, 0.011655011655011656),
 ('胡锡进', 5, 0.011655011655011656),
 ('托腮', 4, 0.009324009324009324),
 ('董扬', 4, 0.009324009324009324),
 ('冯小刚', 4, 0.009324009324009324),
 ('许元', 4, 0.009324009324009324),
 ('老美', 4, 0.009324009324009324),
 ('基辛格', 4, 0.009324009324009324),
 ('里根', 4, 0.009324009324009324),
 ('刘鹤', 4, 0.009324009324009324),
 ('洪博培', 3, 0.006993006993006993),
 ('•罗奇', 3, 0.006993006993006993),
 ('斯蒂芬', 3, 0.006993006993006993),
 ('李克强', 3, 0.006993006993006993),
 ('王', 3, 0.006993006993006993)]


In [56]:

# ['ni'] 组织名
pprint(get_top(posdict['ni'], num=25))


[('路透社', 5, 0.21739130434782608),
 ('外交部', 4, 0.17391304347826086),
 ('国务院', 3, 0.13043478260869565),
 ('北京大学', 2, 0.08695652173913043),
 ('WTO', 2, 0.08695652173913043),
 ('财政部', 2, 0.08695652173913043),
 ('联合国', 1, 0.043478260869565216),
 ('中国银行', 1, 0.043478260869565216),
 ('透社', 1, 0.043478260869565216),
 ('彭博社', 1, 0.043478260869565216),
 ('中共中央', 1, 0.043478260869565216)]


In [57]:

# ['nz'] other proper name
pprint(get_top(posdict['nz'], num=25))


[('华为', 50, 0.15625),
 ('川普', 46, 0.14375),
 ('中兴', 44, 0.1375),
 ('思科', 15, 0.046875),
 ('人民日报', 14, 0.04375),
 ('腾讯', 11, 0.034375),
 ('英语', 11, 0.034375),
 ('WTO', 10, 0.03125),
 ('中华', 10, 0.03125),
 ('腾讯网', 8, 0.025),
 ('英文', 7, 0.021875),
 ('搜狐', 6, 0.01875),
 ('鸦片战争', 5, 0.015625),
 ('联想', 5, 0.015625),
 ('网易', 4, 0.0125),
 ('卡内基', 3, 0.009375),
 ('康师傅', 3, 0.009375),
 ('高通', 3, 0.009375),
 ('星巴克', 3, 0.009375),
 ('优衣库', 3, 0.009375),
 ('富力', 3, 0.009375),
 ('美团', 2, 0.00625),
 ('安卓', 2, 0.00625),
 ('阿里巴巴', 2, 0.00625),
 ('索尼', 2, 0.00625)]


In [59]:

# ['v'] verb len >= 2
# pprint(get_top([x for x in posdict['v'] if len(x[0])>=2], num=25))


In [61]:

# foreign words	
pprint(get_top([x for x in posdict['ws'] if len(x[0])>=4 and not x[0].startswith('http://')], num=25))


[('refund', 8, 0.042328042328042326),
 ('export', 8, 0.042328042328042326),
 ('Twitter', 7, 0.037037037037037035),
 ('Ecup', 6, 0.031746031746031744),
 ('wellness', 6, 0.031746031746031744),
 ('caojitw', 5, 0.026455026455026454),
 ('williamlong', 5, 0.026455026455026454),
 ('list', 4, 0.021164021164021163),
 ('iPhone', 4, 0.021164021164021163),
 ('ICAO', 3, 0.015873015873015872),
 ('license', 3, 0.015873015873015872),
 ('news', 3, 0.015873015873015872),
 ('fake', 3, 0.015873015873015872),
 ('Tabgach', 3, 0.015873015873015872),
 ('Boss', 2, 0.010582010582010581),
 ('even', 2, 0.010582010582010581),
 ("can't", 2, 0.010582010582010581),
 ('wrong', 2, 0.010582010582010581),
 ('freedom', 2, 0.010582010582010581),
 ('philo', 2, 0.010582010582010581),
 ('apple', 2, 0.010582010582010581),
 ('Metoo', 2, 0.010582010582010581),
 ('david20171231', 2, 0.010582010582010581),
 ('bias', 2, 0.010582010582010581),
 ('Harley', 2, 0.010582010582010581)]


In [62]:
# 命名实体
print(nedict.keys())


dict_keys(['O', 'S-Ni', 'S-Ns', 'S-Nh', 'B-Ni', 'E-Ni', 'I-Ni', 'B-Ns', 'I-Ns', 'E-Ns', 'B-Nh', 'E-Nh'])


In [63]:


# 人名
pprint(get_top(nedict['S-Nh'], num=25))


[('特朗普', 28, 0.07466666666666667),
 ('川普', 24, 0.064),
 ('奥巴马', 23, 0.06133333333333333),
 ('孟晚舟', 16, 0.042666666666666665),
 ('崔永元', 8, 0.021333333333333333),
 ('陈德铭', 7, 0.018666666666666668),
 ('崔天凯', 7, 0.018666666666666668),
 ('罗姆尼', 6, 0.016),
 ('张首晟', 6, 0.016),
 ('孙立平', 5, 0.013333333333333334),
 ('胡锡进', 5, 0.013333333333333334),
 ('托腮', 4, 0.010666666666666666),
 ('董扬', 4, 0.010666666666666666),
 ('冯小刚', 4, 0.010666666666666666),
 ('许元', 4, 0.010666666666666666),
 ('老美', 4, 0.010666666666666666),
 ('基辛格', 4, 0.010666666666666666),
 ('里根', 4, 0.010666666666666666),
 ('刘鹤', 4, 0.010666666666666666),
 ('洪博培', 3, 0.008),
 ('李克强', 3, 0.008),
 ('王', 3, 0.008),
 ('蔡英文', 3, 0.008),
 ('老胡', 3, 0.008),
 ('希特勒', 3, 0.008)]


In [64]:

# 机构名
pprint(get_top(nedict['S-Ni'], num=25))


[('路透社', 4, 0.23529411764705882),
 ('国务院', 2, 0.11764705882352941),
 ('外交部', 2, 0.11764705882352941),
 ('WTO', 2, 0.11764705882352941),
 ('财政部', 2, 0.11764705882352941),
 ('联合国', 1, 0.058823529411764705),
 ('中国银行', 1, 0.058823529411764705),
 ('透社', 1, 0.058823529411764705),
 ('彭博社', 1, 0.058823529411764705),
 ('中共中央', 1, 0.058823529411764705)]


In [65]:

# 地名
pprint(get_top(nedict['S-Ns'], num=25))

[('中国', 720, 0.24431625381744146),
 ('美国', 655, 0.22225992534781133),
 ('美', 520, 0.17645062775704107),
 ('中', 443, 0.1503223617237869),
 ('日本', 76, 0.025788937902952155),
 ('欧', 60, 0.02035968781812012),
 ('俄罗斯', 29, 0.009840515778758059),
 ('欧洲', 24, 0.008143875127248049),
 ('加拿大', 19, 0.006447234475738039),
 ('俄', 15, 0.00508992195453003),
 ('非洲', 14, 0.004750593824228029),
 ('山东', 14, 0.004750593824228029),
 ('伊朗', 14, 0.004750593824228029),
 ('韩国', 13, 0.004411265693926026),
 ('朝鲜', 12, 0.0040719375636240245),
 ('英国', 12, 0.0040719375636240245),
 ('北京', 11, 0.0037326094333220224),
 ('香港', 11, 0.0037326094333220224),
 ('南海', 9, 0.003053953172718018),
 ('土耳其', 9, 0.003053953172718018),
 ('台湾', 9, 0.003053953172718018),
 ('日', 9, 0.003053953172718018),
 ('韩', 8, 0.0027146250424160165),
 ('黄海', 8, 0.0027146250424160165),
 ('古巴', 8, 0.0027146250424160165)]


In [66]:
# 以上内容均为自动生成...